In [176]:
import os
import pandas as pd
import numpy as np
from natsort import natsorted
from sklearn import metrics
from sklearn.metrics import classification_report

In [177]:
data_path = "/Users/imrankabir/Desktop/research/vqa_accessibility/Dashboard-For-VQA/Dashboard Data"
gt_fol = os.path.join(data_path, "GT")

In [178]:
coco_common_obj = ['Person', 'Bicycle', 'Car', 'Motorcycle', 'Bus', 'Traffic Signals', 'Fire hydrant', 'Stop sign'
                   'Bench', 'Dog', 'Chair', 'Vegetation']

pfb_common_obj = ['Road', 'Sidewalk', 'Tree', 'Vegetation', 'Building', 'Fence', 'Traffic Signals',
                  'Fire hydrant', 'Chair', 'Trash on roads', 'Trash bins', 'Person', 'Car', 'Motorcycle',
                  'Bus']

com_obj = list(set(coco_common_obj) & set(pfb_common_obj))

len(com_obj)

8

In [179]:
def get_mAP(targets, preds):
    APs = np.zeros(preds.shape[1])
    APs2 = np.zeros(preds.shape[1])
    ARs = np.zeros(preds.shape[1])
    F1s = np.zeros(preds.shape[1])
    ACCs = np.zeros(preds.shape[1])
    for k in range(preds.shape[1]):
        APs[k], ARs[k], F1s[k], ACCs[k] = _average_precision_2(targets[:, k], preds[:, k])
        APs2[k] = _average_precision(targets[:, k], preds[:, k])

    return APs, ARs, F1s, ACCs, APs2

def _average_precision(output, target):
    # print(output, target)
    epsilon = 1e-8

    # sort examples
    indices = output.argsort()[::-1]
    # Computes prec@i
    total_count_ = np.cumsum(np.ones((len(output), 1)))

    target_ = target[indices]

    ind = target_ == 1

    pos_count_ = np.cumsum(ind)

    total = pos_count_[-1]
    pos_count_[np.logical_not(ind)] = 0

    pp = pos_count_ / total_count_
    precision_at_i_ = np.sum(pp)
    # print(pp, precision_at_i_, pos_count_, total_count_)
    precision_at_i = precision_at_i_ / (total + epsilon)

    return precision_at_i


def _average_precision_2(target, output):
    epsilon = 1e-8

    total_pred_pos = np.count_nonzero(output)
    total_target_pos = np.count_nonzero(target)

    TP = np.count_nonzero(np.logical_and(output, target))
    TN = np.count_nonzero(np.logical_and(np.logical_not(output), np.logical_not(target)))
    FP = total_pred_pos - TP
    FN = total_target_pos - TP

    AP = TP / (TP + FP + epsilon)
    AR = TP / (TP + FN + epsilon)
    F1 = (2 * AP * AR) / (AP + AR + epsilon)
    ACC = (TP + FP) / (TP + TN + FP + FN + epsilon)

    return AP, AR, F1, ACC


def calculate_model_map(gt_path, pred_path, obj_list=None):
    gt_dict = {}
    pred_dict = {}
    
    gt_files = [x for x in os.listdir(gt_path) if x.endswith('.csv') and int(x.split('-')[1])<=16]

    for fl in gt_files:
        if fl in ["video-10-segment-1.csv", "video-9-segment-2.csv"]:
            continue
        gt_fl = os.path.join(gt_path, fl)
        pred_fl = os.path.join(pred_path, fl)
        if not os.path.exists(pred_fl):
            continue

        gt_data = pd.read_csv(gt_fl)
        gt_data.replace(-1, 1, inplace=True) 
        x = []
        for index, row in gt_data.iterrows():
            if list(row)[0].strip() in ["Sidewalk pits"]:
                continue
            if list(row)[0].strip() in gt_dict.keys():
                gt_dict[list(row)[0].strip()] += list(row)[1:]
            else:
                gt_dict[list(row)[0].strip()] = list(row)[1:]

            x.append(len(list(row)[1:]))

        y = []    
        pred_data = pd.read_csv(pred_fl)
        pred_data.replace(-1, 1, inplace=True) 
        for index, row in pred_data.iterrows():
            if list(row)[0].strip() in ["Sidewalk pits"]:
                continue
            if list(row)[0].strip() in pred_dict.keys():
                pred_dict[list(row)[0].strip()] += list(row)[1:]
            else:
                pred_dict[list(row)[0].strip()] = list(row)[1:]
            y.append(len(list(row)[1:]))
            
        if x != y:
            print(len(x), len(y), fl)
    
    target_array = []
    pred_array = []
    
    label_names = []

    for key in gt_dict.keys():
        if key in pred_dict.keys():
            if obj_list:
                if key not in obj_list:
                    continue
            if len(gt_dict[key]) == len(pred_dict[key]):
                target_array.append(gt_dict[key])
                pred_array.append(pred_dict[key])
                label_names.append(key)
            else:
                print(len(gt_dict[key]), len(pred_dict[key]), key)

    target_array = np.array(target_array).T
    pred_array = np.array(pred_array).T
    
    try:
        ap_2 = np.array(metrics.precision_score(target_array, pred_array, average=None))
        ap, ar, f1, acc, ap_3 = get_mAP(target_array, pred_array)
        report = classification_report(target_array, pred_array,target_names=label_names)
    except Exception as e:
        # ap = 0
        print(e)
        ap, ar, f1, acc, report = [0], [0], [0], [0], []
        print(target_array.shape, pred_array.shape)
    
    return ap, ar, f1, acc, report, ap_2, ap_3

In [203]:
pred_fol = os.path.join(data_path, "BLIP")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.46      0.60      0.48      1053
   weighted avg       0.89      0.85      0.86      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [205]:
pred_fol = os.path.join(data_path, "GPV-1")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.51      0.55      0.44      1053
   weighted avg       0.89      0.82      0.83      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [206]:
pred_fol = os.path.join(data_path, "yolo_v7")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.67      0.52      0.54      1053
   weighted avg       0.95      0.71      0.72      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/py

In [207]:
pred_fol = os.path.join(data_path, "HRNet_V2")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.30      0.68      0.36      1053
   weighted avg       0.73      0.91      0.79      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/py

In [208]:
pred_fol = os.path.join(data_path, "mask_rcnn")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.39      0.69      0.41      1053
   weighted avg       0.84      0.82      0.78      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [209]:
pred_fol = os.path.join(data_path, "faster_rcnn")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.40      0.70      0.42      1053
   weighted avg       0.85      0.83      0.79      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [210]:
pred_fol = os.path.join(data_path, "Random")
apg, arg, f1g, accg, rep, apg2, apg3 = calculate_model_map(
    gt_fol, pred_fol, 
    obj_list=coco_common_obj
)

rep = rep.split('\n')[0:1] + rep.split('\n')[-4:-2]
rep = "\n".join(rep)
print(rep)
# print(apg.mean(), apg2.mean(), apg3.mean())

                 precision    recall  f1-score   support
      macro avg       0.22      0.43      0.21      1053
   weighted avg       0.64      0.49      0.51      1053


/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/imrankabir/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
